In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
Sistema de Arbitraje Estadístico Bayesiano Adaptativo a Múltiples Regímenes
--------------------------------------------------------------------------------
Este programa descarga datos históricos de precios para los símbolos del S&P500,
realiza un filtrado de pares mediante tests de cointegración (Johansen y KPSS),
estima parámetros bayesianos de forma simplificada, detecta regímenes (modelo HMM),
genera señales de trading adaptativas y ejecuta un backtesting walk‑forward.
Se guardan todas las métricas y gráficos en la carpeta './artifacts/results/'
y se registran errores completos en './artifacts/errors.txt'.
"""

import os
import sys
import logging
import traceback
import datetime
import itertools
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.stattools import kpss
from hmmlearn.hmm import GaussianHMM
from sklearn.cluster import KMeans
from scipy.stats import norm

# =============================================================================
# Configuración de directorios y logging
# =============================================================================

RESULTS_DIR = './artifacts/results'
FIGURES_DIR = os.path.join(RESULTS_DIR, 'figures')
DATA_DIR = os.path.join(RESULTS_DIR, 'data')
ERROR_LOG_FILE = './artifacts/errors.txt'

for folder in [RESULTS_DIR, FIGURES_DIR, DATA_DIR]:
    os.makedirs(folder, exist_ok=True)

logging.basicConfig(
    filename=ERROR_LOG_FILE,
    level=logging.ERROR,
    format='[%(asctime)s] %(levelname)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

def log_exception():
    """Registra el traceback completo de la excepción actual."""
    logging.error("Excepción:\n%s", traceback.format_exc())

# =============================================================================
# Función: Obtener símbolos y datos del S&P500 desde Wikipedia
# =============================================================================

def get_sp500_symbols():
    """
    Descarga la lista de empresas del S&P500 desde Wikipedia.
    Retorna un DataFrame con columnas: Symbol, GICS Sector.
    """
    try:
        url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
        tables = pd.read_html(url)
        df = tables[0]
        df['Symbol'] = df['Symbol'].str.replace('.', '-', regex=False)  # Ajuste para yfinance
        return df[['Symbol', 'GICS Sector']]
    except Exception:
        log_exception()
        return pd.DataFrame(columns=['Symbol', 'GICS Sector'])

# =============================================================================
# Función: Descarga de datos históricos con yfinance
# =============================================================================

def fetch_price_data(symbols, start_date, end_date):
    """
    Descarga datos diarios ajustados (Close) para una lista de símbolos utilizando yfinance.
    Retorna un DataFrame con columnas para cada símbolo.
    """
    data = {}
    for symbol in symbols:
        try:
            df = yf.download(symbol, start=start_date, end=end_date)['Close']
            if df.empty:
                logging.error("Datos vacíos para %s entre %s y %s", symbol, start_date, end_date)
                continue
            data[symbol] = df
            print(f"Descargado: {symbol} ({len(df)} días)")
        except Exception:
            logging.error("Error descargando datos para %s", symbol)
            log_exception()
    
    # Si se descargaron correctamente datos, concatenamos las series (se preserva el índice)
    if data:
        return pd.concat(data, axis=1)
    else:
        return pd.DataFrame()

# =============================================================================
# Función: Test de cointegración (Johansen + KPSS)
# =============================================================================

def cointegration_tests(series1, series2, det_order=0):
    """
    Realiza el test de cointegración de Johansen y KPSS sobre residuos.
    Retorna un diccionario con resultados y parámetros:
      - johansen_stat, johansen_p
      - kpss_stat, kpss_p
      - beta, alpha (de la regresión OLS) y residuos.
    """
    try:
        # Construir matriz para el test de Johansen
        data = np.column_stack([series1, series2])
        result = coint_johansen(data, det_order, k_ar_diff=1)
        trace_stat = result.lr1[0]
        # Se asume cointegración si trace_stat > valor crítico al 1%
        p_val = 0.01 if trace_stat > result.cvt[0, 2] else 0.05
        
        # Calcular parámetros OLS
        X = np.vstack([np.ones(len(series1)), series1]).T
        beta_hat = np.linalg.lstsq(X, series2, rcond=None)[0]
        resid = series2 - X.dot(beta_hat)
        
        # Test KPSS sobre residuos
        kpss_stat, p_value_kpss, _, _ = kpss(resid, nlags="auto")
        
        return {
            'johansen_stat': trace_stat,
            'johansen_p': p_val,
            'kpss_stat': kpss_stat,
            'kpss_p': p_value_kpss,
            'beta': beta_hat[1],
            'alpha': beta_hat[0],
            'resid': resid
        }
    except Exception:
        log_exception()
        return None

# =============================================================================
# Función: Estimación Bayesiana de parámetros con MCMC (Metropolis-Hastings)
# =============================================================================

def bayesian_cointegration(resid, num_iter=5000, burn_in=2000):
    """
    Estima parámetros bayesianos (β) usando un algoritmo Metropolis-Hastings simple.
    Prior: β ~ N(1, 0.5)
    Retorna la media y el intervalo de credibilidad del 95% para β.
    """
    try:
        current_beta = 1.0
        chain = []
        sigma_prop = 0.05
        sigma_lik = np.std(resid) if np.std(resid) > 0 else 1.0
        
        for i in range(num_iter):
            proposal_beta = current_beta + np.random.normal(0, sigma_prop)
            log_prior_current = norm.logpdf(current_beta, loc=1, scale=0.5)
            log_prior_prop = norm.logpdf(proposal_beta, loc=1, scale=0.5)
            
            log_lik_current = np.sum(norm.logpdf(resid, loc=0, scale=sigma_lik))
            log_lik_prop = np.sum(norm.logpdf(resid, loc=0, scale=sigma_lik))
            
            log_accept_ratio = (log_prior_prop + log_lik_prop) - (log_prior_current + log_lik_current)
            if np.log(np.random.rand()) < log_accept_ratio:
                current_beta = proposal_beta
            chain.append(current_beta)
        chain = np.array(chain[burn_in:])
        beta_mean = np.mean(chain)
        beta_ci_low = np.percentile(chain, 2.5)
        beta_ci_high = np.percentile(chain, 97.5)
        return {'beta_mean': beta_mean, 'beta_ci': (beta_ci_low, beta_ci_high)}
    except Exception:
        log_exception()
        return None

# =============================================================================
# Función: Modelo HMM para detección de regímenes
# =============================================================================

def detect_market_regimes(features, n_states=3, n_iter=100):
    """
    Ajusta un modelo HMM gaussiano a las características del mercado.
    Inicializa con KMeans y utiliza Baum-Welch.
    Retorna el modelo ajustado y la secuencia de estados ocultos.
    """
    try:
        X = (features - features.mean()) / features.std()
        kmeans = KMeans(n_clusters=n_states, random_state=42).fit(X)
        init_means = np.array([X[kmeans.labels_ == i].mean(axis=0) for i in range(n_states)])
        
        model = GaussianHMM(n_components=n_states, covariance_type="full", n_iter=n_iter, random_state=42)
        model.means_ = init_means
        model.fit(X)
        hidden_states = model.predict(X)
        return model, hidden_states
    except Exception:
        log_exception()
        return None, None

# =============================================================================
# Función: Generación de señales según Z-score y régimen
# =============================================================================

def generate_signals(resid, half_life, regime, current_date, timestamps):
    """
    Calcula el Z-score usando una ventana móvil y genera señales de entrada/salida.
    Los umbrales dependen del régimen (0: baja, 1: normal, 2: alta volatilidad).
    """
    try:
        window = 30
        resid_series = pd.Series(resid, index=timestamps)
        rolling_mean = resid_series.rolling(window=window, min_periods=1).mean()
        rolling_std = resid_series.rolling(window=window, min_periods=1).std().replace(0, 1)
        last_resid = resid_series.iloc[-1]
        last_mean = rolling_mean.iloc[-1]
        last_std = rolling_std.iloc[-1]
        zscore = (last_resid - last_mean) / last_std
        
        thresholds = {
            0: {"entry": 1.25, "exit": 0.5},
            1: {"entry": 1.75, "exit": 0.75},
            2: {"entry": 2.25, "exit": 1.0}
        }
        regime_threshold = thresholds.get(regime, {"entry": 1.75, "exit": 0.75})
        signal = 0  # 0: sin posición, 1: entrada larga en Y y corta en X, -1: entrada corta en Y y larga en X
        
        if zscore < -regime_threshold["entry"]:
            signal = 1
        elif zscore > regime_threshold["entry"]:
            signal = -1
        if abs(zscore) < regime_threshold["exit"]:
            signal = 0
        
        return {'date': current_date, 'zscore': zscore, 'signal': signal, 'half_life': half_life}
    except Exception:
        log_exception()
        return None

# =============================================================================
# Función: Backtesting y Walk-forward
# =============================================================================

def backtest_walkforward(price_data, pairs, backtest_start, backtest_end):
    """
    Ejecuta el backtesting walk-forward:
      - Ventana train de 2 años y ventana test de 6 meses.
      - Para cada par cointegrado se generan señales y se simula una estrategia simple.
      - Se calculan y guardan métricas (Sharpe, max drawdown, Calmar).
    """
    try:
        results = []
        dates = price_data.index

        # Convertir fechas a datetime
        bt_start = pd.to_datetime(backtest_start)
        bt_end = pd.to_datetime(backtest_end)
        current_train_start = bt_start

        total_windows = 0

        while current_train_start < bt_end:
            train_end = current_train_start + pd.DateOffset(years=2)
            test_end = train_end + pd.DateOffset(months=6)
            train_data = price_data.loc[current_train_start:train_end]
            test_data = price_data.loc[train_end:test_end]
            if train_data.empty or test_data.empty:
                break

            print(f"Procesando ventana: {current_train_start.date()} a {test_end.date()}")
            total_windows += 1

            # Evaluar cada par en esta ventana
            for pair in pairs:
                sym1, sym2 = pair
                try:
                    series1_train = train_data[sym1].dropna()
                    series2_train = train_data[sym2].dropna()
                    common_index = series1_train.index.intersection(series2_train.index)
                    if len(common_index) < 100:
                        continue
                    series1_train = series1_train.loc[common_index]
                    series2_train = series2_train.loc[common_index]

                    coint_result = cointegration_tests(series1_train, series2_train)
                    if coint_result is None:
                        continue
                    if coint_result['johansen_p'] > 0.01:
                        continue

                    # Estimar half-life basado en la aproximación AR(1)
                    resid = coint_result['resid']
                    lagged = resid.shift(1).dropna()
                    current = resid.loc[lagged.index]
                    if len(lagged) < 10:
                        continue
                    phi = np.polyfit(lagged, current, 1)[0]
                    if phi <= 0 or phi >= 1:
                        continue
                    half_life = np.log(0.5) / np.log(phi)
                    if not (1 <= half_life <= 30):
                        continue

                    # Estimación bayesiana de β
                    bayes_est = bayesian_cointegration(resid)
                    if bayes_est is None or (bayes_est['beta_ci'][0] < 0 and bayes_est['beta_ci'][1] > 0):
                        continue

                    # Cálculo de residuos en test usando parámetros OLS
                    test_common_index = test_data.index
                    if sym1 not in test_data or sym2 not in test_data:
                        continue
                    X_test = np.vstack([np.ones(len(test_data)), test_data[sym1]]).T
                    resid_test = test_data[sym2] - X_test.dot([coint_result['alpha'], coint_result['beta']])
                    
                    # Generar señales diarias en el período de test
                    signals = []
                    for current_date in resid_test.index:
                        s = generate_signals(resid_test.loc[:current_date].values, half_life, regime=1, 
                                               current_date=current_date, timestamps=resid_test.loc[:current_date].index)
                        if s is not None:
                            signals.append(s)
                    
                    if len(signals) == 0:
                        continue
                    # Simular la estrategia: acumulación diaria de pnl
                    position = 0
                    pnl = []
                    for s in signals:
                        if s['signal'] != 0:
                            position = s['signal']
                        else:
                            position = 0
                        # Se calcula retorno diario a partir de % cambio en sym2
                        ret = test_data[sym2].pct_change().get(s['date'], 0)
                        pnl.append(position * ret)
                    pnl_series = pd.Series(pnl, index=[s['date'] for s in signals])
                    cum_return = (1 + pnl_series).cumprod() - 1
                    sharpe = pnl_series.mean() / pnl_series.std() * np.sqrt(252) if pnl_series.std() > 0 else 0
                    max_dd = (cum_return.cummax() - cum_return).max()
                    calmar = (cum_return.iloc[-1] / max_dd) if max_dd != 0 else 0

                    results.append({
                        'pair': f"{sym1}-{sym2}",
                        'train_start': current_train_start.strftime('%Y-%m-%d'),
                        'train_end': train_end.strftime('%Y-%m-%d'),
                        'test_end': test_end.strftime('%Y-%m-%d'),
                        'sharpe': sharpe,
                        'max_drawdown': max_dd,
                        'calmar': calmar
                    })
                except Exception:
                    log_exception()
                    continue
            current_train_start = test_end
        if total_windows == 0:
            print("No se pudo procesar ninguna ventana walk-forward. Verifique el rango de fechas y datos.")
        else:
            print(f"Se procesaron {total_windows} ventanas de backtesting.")
        # Guardar resultados
        results_df = pd.DataFrame(results)
        results_csv = os.path.join(DATA_DIR, 'performance_metrics.csv')
        results_df.to_csv(results_csv, index=False)
        print(f"Métricas guardadas en: {results_csv}")
        
        # Gráfico de distribución de Sharpe ratios (si hay datos)
        if not results_df.empty:
            plt.figure(figsize=(8, 5))
            plt.hist(results_df['sharpe'].dropna(), bins=20, edgecolor='black')
            plt.title('Distribución de Sharpe Ratios')
            plt.xlabel('Sharpe Ratio')
            plt.ylabel('Frecuencia')
            plt.tight_layout()
            figure_path = os.path.join(FIGURES_DIR, 'sharpe_distribution.png')
            plt.savefig(figure_path)
            plt.close()
            print(f"Gráfico guardado en: {figure_path}")
        else:
            print("No se generaron métricas de performance en el backtesting.")
    except Exception:
        log_exception()

# =============================================================================
# Función principal
# =============================================================================

def main():
    try:
        # Definir periodo amplio para backtesting (2000 hasta hoy)
        end_date = datetime.datetime.today().strftime('%Y-%m-%d')
        start_date = '2000-01-01'
        
        print("Obteniendo símbolos del S&P500...")
        sp500_df = get_sp500_symbols()
        symbols = sp500_df['Symbol'].tolist()
        if not symbols:
            logging.error("No se pudieron obtener símbolos del S&P500")
            sys.exit("Error: No se obtuvieron símbolos.")
        
        print(f"Se obtuvieron {len(symbols)} símbolos.")
        print("Descargando datos históricos...")
        price_data = fetch_price_data(symbols, start_date, end_date)
        if price_data.empty:
            logging.error("No se pudieron descargar datos de precios.")
            sys.exit("Error: Datos de precios vacíos.")
        
        # Crear pares basados en sector para diversificar
        pairs = []
        sectors = sp500_df.groupby('GICS Sector')
        for sector, group in sectors:
            syms = group['Symbol'].tolist()
            for pair in itertools.combinations(syms, 2):
                pairs.append(pair)
        if len(pairs) < 30:
            logging.error("Número insuficiente de pares evaluados")
            sys.exit("Error: No hay suficientes pares.")
        
        print(f"Total de pares a evaluar: {len(pairs)}")
        # Ejecutar backtesting walk-forward
        print("Iniciando backtesting walk-forward...")
        backtest_walkforward(price_data, pairs, start_date, end_date)
        print("Ejecución completada. Revise la carpeta './artifacts/results/' para métricas y gráficos.")
    except Exception:
        log_exception()
        sys.exit("Error en la ejecución del programa.")

if __name__ == '__main__':
    main()


Obteniendo símbolos del S&P500...
Se obtuvieron 503 símbolos.
Descargando datos históricos...


[*********************100%***********************]  1 of 1 completed


Descargado: MMM (6359 días)


[*********************100%***********************]  1 of 1 completed


Descargado: AOS (6359 días)


...

Descargado: ZBRA (6359 días)


[*********************100%***********************]  1 of 1 completed


Descargado: ZBH (5966 días)


[*********************100%***********************]  1 of 1 completed


Descargado: ZTS (3069 días)
Total de pares a evaluar: 13486
Iniciando backtesting walk-forward...
Procesando ventana: 2000-01-01 a 2002-07-01
Procesando ventana: 2002-07-01 a 2005-01-01
Procesando ventana: 2005-01-01 a 2007-07-01
Procesando ventana: 2007-07-01 a 2010-01-01
Procesando ventana: 2010-01-01 a 2012-07-01
Procesando ventana: 2012-07-01 a 2015-01-01
Procesando ventana: 2015-01-01 a 2017-07-01
Procesando ventana: 2017-07-01 a 2020-01-01
Procesando ventana: 2020-01-01 a 2022-07-01
Procesando ventana: 2022-07-01 a 2025-01-01
Se procesaron 10 ventanas de backtesting.
Métricas guardadas en: ./artifacts/results/data/performance_metrics.csv
No se generaron métricas de performance en el backtesting.
Ejecución completada. Revise la carpeta './artifacts/results/' para métricas y gráficos.


In [3]:
print("DONE")

DONE
